In [1]:
!pip install PySastrawi

In [2]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\dzikr\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [3]:
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
factory = StemmerFactory()
stemmer = factory.create_stemmer()

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
import random
import json
import pickle
from tensorflow.keras.callbacks import EarlyStopping
import nltk
from sklearn.model_selection import KFold
from tensorflow.keras import layers, models, callbacks, optimizers
import numpy as np

In [4]:
# Load the intents file
with open('new.json') as data_file:
    intents = json.load(data_file)

In [5]:
# Download the 'punkt_tab' data package
nltk.download('punkt_tab')

# Pre-processing
words = []
classes = []
documents = []
ignore = ['!', '"', "'", '(', ')', ',', '-', '.', ':', ';', '?', '[', ']', '_',
          'adalah', 'akan', 'aku', 'anda', 'atau', 'dalam', 'dan',
          'dari', 'dengan', 'di', 'dia', 'harus', 'ini', 'itu', 'jika', 'kami', 'kamu', 'ke',
          'kita', 'mereka', 'oleh', 'pada', 'saya', 'sebuah', 'sedang', 'sementara', 'tanpa',
          'tapi', 'telah', 'untuk', 'yang', '{', '}']



#tokenize
for intent in intents:
    for pattern in intent['patterns']:
        w = nltk.word_tokenize(pattern.lower())  # Tokenizing and converting to lowercase
        words.extend([word for word in w if word not in ignore])  # Exclude words in ignore list
        documents.append((w, intent['tag']))
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\dzikr\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [6]:
# Melakukan stemming dan normalisasi data
words = [stemmer.stem(w.lower()) for w in words if w not in ignore]
words = sorted(list(set(words)))

# Menghapus class duplikat dengan 'set'
classes = sorted(list(set(classes)))

print(len(documents), "documents")
print(len(classes), "classes", classes)
print(len(words), "unique stemmed words", words)

965 documents
47 classes ['apa_itu_gangguan_mental', 'badmood_dampak', 'badmood_efek_jangka_panjang', 'badmood_gejala', 'badmood_pengobatan', 'badmood_penyebab', 'badmood_preventif', 'badmood_solusi_pribadi', 'badmood_terhadap_kesehatan_fisik', 'badmood_terhadap_pekerjaan', 'bunuh_diri', 'cara_mengatasi_gangguan_mental', 'diagnosis_gangguan_mental', 'dukungan_mental_terapi', 'efek_gangguan_mental', 'gangguan_mental_dan_bipolar', 'gangguan_mental_dan_depresi', 'gangguan_mental_dan_kecemasan', 'gangguan_mental_dan_psikoedukasi', 'gangguan_mental_dan_psikoterapi', 'gangguan_mental_dan_stigma', 'gangguan_mental_untuk_anak', 'gejala_gangguan_mental', 'kabar_baik', 'kabar_buruk', 'kabar_netral', 'mood_bahagia', 'mood_cemas', 'mood_marahan', 'mood_membaik', 'mood_motivation', 'mood_sedih', 'mood_stress_relief', 'penanganan_gangguan_mental', 'pencegahan_gangguan_mental', 'pengobatan_gangguan_mental', 'penyebab_gangguan_mental', 'perbedaan_gangguan_mental_dan_stres', 'perkenalan_diri', 'perpisa

In [7]:
import random
import numpy as np

training = []
output_empty = [0] * len(classes)

for doc in documents:
    pattern_words = [stemmer.stem(word.lower()) for word in doc[0]]
    bag = [1 if w in pattern_words else 0 for w in words]

    output_row = output_empty[:]
    output_row[classes.index(doc[1])] = 1

    training.append([bag, output_row])

random.shuffle(training)
training = np.array(training, dtype=object)

train_x = np.array([item[0] for item in training])
train_y = np.array([item[1] for item in training])


## MODEL

In [8]:
model = models.Sequential([
        layers.Input(shape=(len(train_x[0]),), name='input_layer'),
        layers.Dense(256, activation='relu', kernel_regularizer='l2', name='hidden_layer1'),
        layers.BatchNormalization(),
        layers.Dropout(0.3, name='dropout1'),
        layers.Dense(128, activation='relu', kernel_regularizer='l2', name='hidden_layer2'),
        layers.BatchNormalization(),
        layers.Dropout(0.3, name='dropout2'),
        layers.Dense(len(train_y[0]), activation='softmax', name='output_layer')
    ])

In [9]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ hidden_layer1 (Dense)                │ (None, 256)                 │         113,152 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization                  │ (None, 256)                 │           1,024 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout1 (Dropout)                   │ (None, 256)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ hidden_layer2 (Dense)                │ (None, 128)                 │          32,896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_1                │ (None, 128)                 │             512 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout2 (Dropout)                   │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ output_layer (Dense)                 │ (None, 47)                  │           6,063 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 153,647 (600.18 KB)

 Trainable params: 152,879 (597.18 KB)

 Non-trainable params: 768 (3.00 KB)

In [10]:
early_stopping_callback = callbacks.EarlyStopping(patience=10, restore_best_weights=True)

## K_FOLD

In [11]:
tf.compat.v1.reset_default_graph()

# Hyperparameters
num_folds = 5
batch_size = 8
learning_rate = 1e-4
epochs = 100


train_x = np.array(train_x)
train_y = np.array(train_y)


kf = KFold(n_splits=num_folds, shuffle=True, random_state=42)
fold_accuracies = []

for fold, (train_index, val_index) in enumerate(kf.split(train_x)):
    print(f"\n=== Training Fold {fold + 1}/{num_folds} ===")
    
    X_train, X_val = train_x[train_index], train_x[val_index]
    y_train, y_val = train_y[train_index], train_y[val_index]

    tensorboard_callback = callbacks.TensorBoard(log_dir=f'./logs/fold_{fold+1}')
    early_stopping_callback = callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
    
    optimizer = optimizers.Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

    history = model.fit(
        X_train, y_train,
        validation_data=(X_val, y_val),
        epochs=epochs,
        batch_size=batch_size,
        callbacks=[tensorboard_callback, early_stopping_callback],
        verbose=1
    )

    val_loss, val_accuracy = model.evaluate(X_val, y_val, verbose=0)
    print(f"Fold {fold + 1} - Validation Accuracy: {val_accuracy:.4f}")
    fold_accuracies.append(val_accuracy)
    
avg_accuracy = np.mean(fold_accuracies)
print(f"\n=== Average Validation Accuracy: {avg_accuracy:.4f} ===")



=== Training Fold 1/5 ===
Epoch 1/100
97/97 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - accuracy: 0.0297 - loss: 9.5115 - val_accuracy: 0.0155 - val_loss: 8.2621
Epoch 2/100
97/97 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.0453 - loss: 8.6411 - val_accuracy: 0.0155 - val_loss: 7.9838
Epoch 3/100
97/97 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.0693 - loss: 8.1164 - val_accuracy: 0.0363 - val_loss: 7.6542
Epoch 4/100
97/97 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.1003 - loss: 7.4918 - val_accuracy: 0.0881 - val_loss: 7.2704
Epoch 5/100
97/97 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.1344 - loss: 7.0643 - val_accuracy: 0.1295 - val_loss: 6.8518
Epoch 6/100
97/97 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.1888 - loss: 6.7148 - val_accuracy: 0.2228 - val_loss: 6.4643
Epoch 7/100
97/97 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.1818 - loss: 6.5483 - val_accuracy: 0.2953 - val_loss: 6.1742
Epoch 8/100
97/97 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.2917 - loss: 6.116

## Callbacks

## Training tanpa split

In [12]:
model.compile(
    optimizer=optimizers.Adam(learning_rate=1e-4),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Train and save the final model
model.fit(train_x, train_y, epochs=100, batch_size=8, callbacks=[tensorboard_callback, early_stopping_callback], verbose=1)
model.save('final_model_trained_on_full_data.h5')

Epoch 1/100
121/121 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.8873 - loss: 1.1281
Epoch 2/100
 63/121 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8554 - loss: 1.1682

C:\Users\dzikr\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\callbacks\early_stopping.py:155: UserWarning: Early stopping conditioned on metric `val_loss` which is not available. Available metrics are: accuracy,loss
  current = self.get_monitor_value(logs)


121/121 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8576 - loss: 1.1474
Epoch 3/100
121/121 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8716 - loss: 1.0758
Epoch 4/100
121/121 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9032 - loss: 1.0269
Epoch 5/100
121/121 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8839 - loss: 1.0117
Epoch 6/100
121/121 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9031 - loss: 0.9442
Epoch 7/100
121/121 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9180 - loss: 0.9036
Epoch 8/100
121/121 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9035 - loss: 0.9113
Epoch 9/100
121/121 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9108 - loss: 0.9096
Epoch 10/100
121/121 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9322 - loss: 0.8567
Epoch 11/100
121/121 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9222 - loss: 0.8650
Epoch 12/100
121/121 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9437 - loss: 0.8242
Epoch 13/100
121/121 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/ste

In [13]:
import json

# Save
try:
    with open("training_data.json", "w") as file:
        # Convert non-serializable data like NumPy arrays or other objects to lists
        data = {'words': words, 'classes': classes, 'train_x': train_x.tolist(), 'train_y': train_y.tolist()}
        json.dump(data, file)
except IOError as e:
    print(f"Error saving training data: {e}")

In [14]:
# Save
try:
    with open("training_data", "wb") as file:
        pickle.dump({'words': words, 'classes': classes, 'train_x': train_x, 'train_y': train_y}, file)
except IOError as e:
    print(f"Error saving training data: {e}")

In [15]:
# Load
try:
    with open("training_data", "rb") as file:
        data = pickle.load(file)

    words = data['words']
    classes = data['classes']
    train_x = np.array(data['train_x'])
    train_y = np.array(data['train_y'])
except IOError as e:
    print(f"Error loading training data: {e}")

# Load model
try:
    model = keras.models.load_model('final_model_trained_on_full_data.h5')
except IOError as e:
    print(f"Error loading model: {e}")


In [16]:
def clean_up_sentence(sentence):
    sentence_words = nltk.word_tokenize(sentence)
    sentence_words = [stemmer.stem(word.lower()) for word in sentence_words]
    return sentence_words

def bow(sentence, words, show_details=False):
    sentence_words = clean_up_sentence(sentence)
    bag = np.zeros(len(words), dtype=np.float32)
    for idx, w in enumerate(words):
        if w in sentence_words:
            bag[idx] = 1
            if show_details:
                print(f"Found in bag: {w}")
    return bag

ERROR_THRESHOLD = 0.70



def classify(sentence):
    input_data = bow(sentence, words)
    results = model.predict(np.array([input_data]))[0]
    results = [[i, r] for i, r in enumerate(results) if r >= ERROR_THRESHOLD]
    results.sort(key=lambda x: x[1], reverse=True)
    return [(classes[r[0]], r[1]) for r in results]

def response(sentence, userID='123', show_details=False):
    results = classify(sentence)
    if results:
        for intent in intents:
            if intent['tag'] == results[0][0]:
                return random.choice(intent['responses'])
    return "Maaf, saya belum mengerti apa yang Anda bicarakan, perlu bantuan?."

In [19]:
print("0 to close")
while True:
    message = input("")
    if message == "0":
        break
    result = response(message)

    if result is not None and "~" in result:
        order = (result[1:])
        action(order)
    else:
        print(result)


0 to close


 0


In [21]:
model = tf.keras.models.load_model('final_model_trained_on_full_data.h5')

converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()


with open('model.tflite', 'wb') as f:
  f.write(tflite_model)

INFO:tensorflow:Assets written to: C:\Users\dzikr\AppData\Local\Temp\tmpiossyh0y\assets


INFO:tensorflow:Assets written to: C:\Users\dzikr\AppData\Local\Temp\tmpiossyh0y\assets


Saved artifact at 'C:\Users\dzikr\AppData\Local\Temp\tmpiossyh0y'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 441), dtype=tf.float32, name='input_layer')
Output Type:
  TensorSpec(shape=(None, 47), dtype=tf.float32, name=None)
Captures:
  2601258342480: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2601258344976: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2601258345360: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2601258343632: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2601258344208: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2601258346128: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2601258344400: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2601258348240: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2601258347088: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2601258347856: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2601258347472